In [ ]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

In [ ]:
!pip install snowflake-connector-python


In [1]:
from snowflake.snowpark import Session

In [2]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])
    print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [3]:
session = get_session()
# session.close()
session

connection_params =  {'user': 'PRAKHAR', 'password': 'Prak@apr24', 'account': 'ug94937.us-east4.gcp', 'warehouse': 'FOSFOR_INSIGHT_WH', 'database': 'test_database_prak', 'schema': 'test_schema_prak', 'role': 'PRAKHAR'}


In [4]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWSIGHT"

In [5]:
# from snowflake.ml.registry import model_registry
from snowflake.ml.registry import Registry as model_registry

In [6]:
!pip freeze | grep -i cloud

cloudpickle==3.0.0


In [ ]:
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [7]:
registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
# registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [13]:
df = registry.show_models()
type(df)

pandas.core.frame.DataFrame

In [ ]:
import cloudpickle

model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
model_obj

In [ ]:
x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
x_train

In [ ]:
mv = registry.log_model(modmodel_obj,
                   model_name="my_model",
                   version_name="v1",
                   conda_dependencies=["scikit-learn"],
                   comment="My awesome ML model",
#                    metrics={"score": 96},
                   sample_input_data=x_train)